In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


In [ ]:
import numpy as np 
import pandas as pd 
import os 

print(os.listdir('../input'))


In [ ]:
train_df =  pd.read_csv('/kaggle/input/new-york-city-taxi-fare-prediction/train.csv', nrows = 10_000_000)
test_df =  pd.read_csv('/kaggle/input/new-york-city-taxi-fare-prediction/test.csv')
print('Training data size: %d' % len(train_df))
print('Test data size: %d' % len(test_df))


In [ ]:
train_df.head()


In [ ]:
train_df.dtypes


In [ ]:
# import seaborn as sns
# # გამოვთვალოთ რა დამოკიდებულება აქვს fair_amount-თან სხვადასხვა features
# correlation_matrix = train_df.corr()

# # ავაგოთ შესაბამისი გრაფი
# plt.figure(figsize=(12, 8))
# sns.heatmap(correlation_matrix[['fare_amount']], annot=True, cmap='coolwarm', linewidths=0.5)
# plt.title('Correlation between Features and Fare Amount')
# plt.show()


In [ ]:
print(train_df.isnull().sum())


In [ ]:
print(test_df.isnull().sum())


In [ ]:
train_df = train_df.dropna(how = 'any', axis = 'rows')
print(train_df.isnull().sum())


In [ ]:
train_df.describe()


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.hist(train_df['fare_amount'], bins=50, color='skyblue', edgecolor='black')
plt.xlabel('Fare Amount')
plt.ylabel('Frequency')
plt.title('Distribution of Fare Amount (0 to 200)')
plt.grid(True)
plt.show()


In [ ]:
train_df = train_df[train_df.fare_amount > 2.5]


In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(train_df['fare_amount'], bins=50, range=(0, 200), color='skyblue', edgecolor='black')
plt.xlabel('Fare Amount')
plt.ylabel('Frequency')
plt.title('Distribution of Fare Amount (0 to 200)')
plt.grid(True)
plt.show()


In [ ]:
train_df['fare_amount'].max()  # მაქსიმალური და საშუალო რომ ვნახოთ შევაფასოთ საშუალოდ რა არის ხოლმე გადასახადი
train_df['fare_amount'].mean() 


In [ ]:
passenger_counts = train_df['fare_amount'].value_counts()
fair_amount_mean = train_df['fare_amount'].mean()*30


In [ ]:
plt.figure(figsize=(10, 6))
passenger_counts.plot(kind='bar', color='skyblue', edgecolor='black')
plt.xlabel('Fare Amount')
plt.ylabel('Frequency')
plt.title('Distribution of Fare Amount (0 to 200)')
plt.xticks(rotation=0)  
plt.grid(axis='y')  
plt.xlim(2.5, fair_amount_mean)
plt.show()


In [ ]:
train_df = train_df[train_df.fare_amount < fair_amount_mean]


In [ ]:
pd.set_option('float_format', '{:f}'.format)
train_df.describe()


In [ ]:
min(test_df.pickup_longitude.min(), test_df.dropoff_longitude.min()), \
max(test_df.pickup_longitude.max(), test_df.dropoff_longitude.max())


In [ ]:
min(test_df.pickup_latitude.min(), test_df.dropoff_latitude.min()), \
max(test_df.pickup_latitude.max(), test_df.dropoff_latitude.max())


In [ ]:
train_df = train_df.loc[train_df.pickup_latitude > 40.2]
train_df = train_df.loc[train_df.pickup_latitude < 41.8]


In [ ]:
train_df = train_df.loc[train_df.dropoff_latitude > 40.2]
train_df = train_df.loc[train_df.dropoff_latitude < 41.8]


In [ ]:
train_df = train_df.loc[train_df.dropoff_longitude < -72]
train_df = train_df.loc[train_df.dropoff_longitude > -74.5]


In [ ]:
train_df = train_df.loc[train_df.pickup_longitude < -72]
train_df = train_df.loc[train_df.pickup_longitude > -74.5]


In [ ]:
train_df.shape


In [ ]:
train_df.describe()


In [ ]:
passenger_counts = train_df['passenger_count'].value_counts()

# Plot bar plot
plt.figure(figsize=(10, 6))
passenger_counts.plot(kind='bar', color='skyblue', edgecolor='black')
plt.xlabel('Number of Passengers')
plt.ylabel('Number of Rides')
plt.title('Number of Rides by Number of Passengers')
plt.xticks(rotation=0)  # Rotate x-axis labels
plt.grid(axis='y')  # Add gridlines to y-axis
plt.show()


In [ ]:
train_df = train_df.loc[train_df.passenger_count > 0]
train_df = train_df.loc[train_df.passenger_count < 7]


In [ ]:
train_df.shape


In [ ]:
merged_df = pd.concat([train_df,test_df], axis = 0, sort=False)
merged_df


In [ ]:
merged_df.shape


In [ ]:
merged_df.isna().sum()
# test_df.isna().sum()
# test_df.shape


In [ ]:
from haversine import haversine
def haversine_distance(row):
    pickup_coord = (row['pickup_latitude'], row['pickup_longitude'])
    dropoff_coord = (row['dropoff_latitude'], row['dropoff_longitude'])
    return haversine(pickup_coord, dropoff_coord)

merged_df['haversine_distance'] = merged_df.apply(haversine_distance, axis=1)


In [ ]:
from pandas.tseries.holiday import USFederalHolidayCalendar
cal = USFederalHolidayCalendar()
holidays = cal.holidays(start='2009-01-01', end='2015-12-31').to_pydatetime()


merged_df['pickup_datetime'] = pd.to_datetime(merged_df['pickup_datetime']).dt.tz_localize(None)
holidays = [holiday.replace(tzinfo=None) for holiday in holidays]  
holidays = pd.to_datetime(holidays, utc=True).tz_convert(merged_df['pickup_datetime'].dt.tz) 

# train_df['pickup_datetime'] = pd.to_datetime( train_df['pickup_datetime'])
merged_df['is_holiday'] = merged_df['pickup_datetime'].isin(holidays).astype(int)
num_holidays = (merged_df['is_holiday'] == 1).sum()
print("Number of holiday rows:", num_holidays)


In [ ]:
date_feature = merged_df['pickup_datetime']
year = date_feature.dt.year
month = date_feature.dt.month
day = date_feature.dt.day
hour = date_feature.dt.hour
minute = date_feature.dt.minute
second = date_feature.dt.second
merged_df['year'] = year
merged_df['month'] = month
merged_df['day'] = day
merged_df['hour'] = hour
merged_df['minute'] = minute
merged_df['second'] = second


In [ ]:
merged_df.drop(['pickup_datetime'], axis = 1, inplace = True)


In [ ]:
until = train_df.shape[0]
train = merged_df.iloc[0:until, :]
test = merged_df.iloc[until:, :] 
test.drop('fare_amount', axis = 1, inplace = True) 


In [ ]:
train.head()


In [ ]:
test.head()


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
import xgboost


In [ ]:
# import seaborn as sns
# # გამოვთვალოთ რა დამოკიდებულება აქვს fair_amount-თან სხვადასხვა features
# correlation_matrix = train.corr()

# # ავაგოთ შესაბამისი გრაფი
# plt.figure(figsize=(12, 8))
# sns.heatmap(correlation_matrix[['fare_amount']], annot=True, cmap='coolwarm', linewidths=0.5)
# plt.title('Correlation between Features and Fare Amount')
# plt.show()


In [ ]:
target=train['fare_amount']


In [ ]:
x_train, x_valid , y_train, y_valid = train_test_split(train, target, test_size=.2, random_state=0)

x_train.shape, y_train.shape, x_valid.shape, y_valid.shape


In [ ]:
train.isna().sum()


In [ ]:
# target.shape
# x_train.shape


In [ ]:
target = x_train['fare_amount']
x_train = x_train.drop(['fare_amount'], axis = 1)


In [ ]:
# target = target.dropna(how = 'any', axis = 'rows')
# target.isnull().sum()
# x_train.isna().sum()


In [ ]:
test_keys = test['key']
x_train.drop(['key'], axis = 1, inplace=True)


In [ ]:
test_keys.head()


In [ ]:
test_x = test
test_x.drop(['key'], axis= 1, inplace=True)


In [ ]:
# train_y = train.fare_amount
# x_train = x_train.drop(['fare_amount'], axis = 1)

model = LinearRegression()
model.fit(x_train, target)

# # Make prediction
y_pred = model.predict(test_x)


In [ ]:
submission = pd.DataFrame(
    {'key': test_keys, 'fare_amount': y_pred},
    columns = ['key', 'fare_amount'])
submission.to_csv('submission.csv', index = False)

print(os.listdir('.'))


In [ ]:
model.coef_


In [ ]:
expected_valid = x_valid['fare_amount']
expected_valid


In [ ]:
x_valid.drop(['fare_amount'], axis=1, inplace=True)
x_valid.head()


In [ ]:
# valid_pred = model.predict(x_valid)


In [ ]:
# mse = mean_squared_error(expected_valid, valid_pred)
# print("MSE  --> ", mse)


In [ ]:
# import math
# rmse = math.sqrt(mse)
# print("RMSE  --> ", rmse)


In [ ]:
# plt.scatter(expected_valid, valid_pred)
# plt.xlabel("Actual")
# plt.ylabel("Predicted")


In [ ]:
# import seaborn as sns
# sns.regplot(x=expected_valid, y=valid_pred, ci=None, color='purple')


In [ ]:
# new_train = x_train
# new_train.drop(['passenger_count', 'second', 'minute', 'day', 'hour', 'month', 'year'], axis=1, inplace=True)


In [ ]:
# new_train.drop(['dropoff_latitude', 'dropoff_longitude'], axis=1, inplace=True)


In [ ]:
# new_valid = x_valid
# new_valid.drop(['passenger_count', 'second', 'minute', 'day', 'hour', 'month', 'year'], axis=1, inplace=True)


In [ ]:
# new_valid.drop(['dropoff_latitude', 'dropoff_longitude'], axis=1, inplace=True)


In [ ]:
# new_model = LinearRegression()
# model.fit(new_train, target)

# # # Make prediction
# new_pred = model.predict(new_valid)
# display(new_pred)


In [ ]:
# mse = mean_squared_error(expected_valid, valid_pred)
# print("MSE  --> ", mse)
# rmse = math.sqrt(mse)
# print("RMSE  --> ", rmse)
# plt.scatter(expected_valid, new_pred)
# plt.xlabel("Actual")
# plt.ylabel("Predicted")


In [ ]:
# new_pred.mean()
# train_df.mean()


In [ ]:
# expected_valid.head()


In [ ]:
# from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import mean_squared_error
# from sklearn.svm import SVR
# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestRegressor
# import xgboost


In [ ]:
# sc = StandardScaler()
# train_scaled = sc.fit_transform(train_df)
# train_scaled[:5]


In [ ]:
# train_df.head()


In [ ]:
# new_train_scaled = pd.DataFrame(train_scaled, columns=['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'passenger_count', 'is_holiday', 'year', 'month', 'day', 'hour', 'minute', 'second', 'haversine_distance'])
# new_train_scaled.to_csv('.//scaled_train_df.csv', index=False )


In [ ]:
# x_train, x_valid , y_train, y_valid = train_test_split(train_df, target, test_size=.2)

# x_train.shape, y_train.shape, x_valid.shape, y_valid.shape


In [ ]:
# from sklearn.linear_model import LinearRegression


In [ ]:
# (w, _, _, _) = np.linalg.lstsq(x_train, y_train, rcond = None)
# print(w)


In [ ]:
# w_OLS = np.matmul(np.matmul(np.linalg.inv(np.matmul(x_train.T, x_train)), x_train.T), y_train)
# print(w_OLS)


In [ ]:
# test_df.head()


In [ ]:
# test_df['year'].min() 
# test_df['year'].max()


In [ ]:
# cal = USFederalHolidayCalendar()
# holidays = cal.holidays(start='2009-01-01', end='2015-12-31').to_pydatetime()


# test_df['pickup_datetime'] = pd.to_datetime(test_df['pickup_datetime']).dt.tz_localize(None)
# holidays = [holiday.replace(tzinfo=None) for holiday in holidays]  
# holidays = pd.to_datetime(holidays, utc=True).tz_convert(test_df['pickup_datetime'].dt.tz) 

# # train_df['pickup_datetime'] = pd.to_datetime( train_df['pickup_datetime'])
# test_df['is_holiday'] = test_df['pickup_datetime'].isin(holidays).astype(int)
# num_holidays = (test_df['is_holiday'] == 1).sum()
# print("Number of holiday rows:", num_holidays)


In [ ]:
# test_date_feature = test_df['pickup_datetime']
# year = test_date_feature.dt.year
# month = test_date_feature.dt.month
# day = test_date_feature.dt.day
# hour = test_date_feature.dt.hour
# minute = test_date_feature.dt.minute
# second = test_date_feature.dt.second
# test_df['year'] = year
# test_df['month'] = month
# test_df['day'] = day
# test_df['hour'] = hour
# test_df['minute'] = minute
# test_df['second'] = second


In [ ]:
# target_t = test_df['key']


In [ ]:
# test_df.drop(['key'], axis=1, inplace=True)


In [ ]:
# test_df.drop([ 'pickup_datetime'], axis = 1, inplace=True)


In [ ]:
# test_df['haversine_distance'] = test_df.apply(haversine_distance, axis=1)


In [ ]:
# sc = StandardScaler()
# test_scaled = sc.transform(test_df)
# test_scaled[:5]
